In [ ]:
#pip install tensorflow==1.13.1
#pip install keras==2.2.4
!pip install emoji
!pip install fasttext
!pip install unidecode
!pip install gensim==3.8.3

In [ ]:
import numpy as np
import emoji
import re
import string
import fasttext
import fasttext.util
from gensim.models.wrappers import FastText
from gensim.models import KeyedVectors, TranslationMatrix
from gensim.models.wrappers import FastText


import keras
from keras.utils import to_categorical
import numpy as np
import pickle

from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from xml.dom import minidom
from string import punctuation
import sys,string
class Prepocessing:
    def __init__(self,with_gensim=True,path_to_data=""):
        """@path_to_data represent the dict where are store pickle object who 
            contain data, sentences, tokens"""
        self.my_punctuation=string.punctuation+"»«'´’…‘”“,¨¦´ʼʽʾʿˀˁ͵ͺ΄ʻ·"
        self.data=[]
        self.sentences=[]
        self.all_sentences=[]
        self.test_sentences=[]
        self.embedding={}
        self.tokenizer={}
        self.rever_tokenizer={}
        self.vocab_size=0
        self.max_length=0
        self.word2vec={}
        if(path_to_data!=""):
            with open(path_to_data,"rb") as f:
                temp=pickle.loads(f.read())
                self.data=temp["data"]
                self.sentences=temp["sentences"]
                self.vocab=temp["tokens"]
    
    def clean_emoji(self,text):
      '''
        remove all of emojis from text
        -------------------------
      '''
      elt_2=self.replace_all_shorcuts(text)
      elt_2=self.replace_punctuaion(elt_2,string.punctuation+"»«'´’…‘”“,¨¦´ʼʽʾʿˀˁ͵ͺ΄ʻ·")
      text=  emoji.demojize(elt_2)
      text= re.sub(r'(:[!_\-\w]+:)', '', text)
      return text
    
    def load_txt_sentences(self,file_path):
      """Charge les donnees d'un fichier texte sous forme de phrase"""

      with open(file_path) as f:
        data=f.read()
        data=data.split("\n")
      return data          
        
#        if(with_gensim==True):
#            self.word2vec=gensim.models.KeyedVectors.load_word2vec_format("word_representation/google_300_wtv.bin", binary=True)
#        else:
#            with open("word_representation/personnal_embedding.bin","rb") as f:
#                self.word2vec=pickle.loads(f.read())
            

    def get_word_representation(self,word):
        try:
            array_rep=self.word2vec[word.lower()]
            return array_rep
        except:
            return np.random.uniform(-0.25, 0.25, 300)
    
    def load_xml_data_with_dom(self,path_to_data,test=False,with_punc=False):
        data=""
        with open(path_to_data,"r") as f:
            data=f.read()
        parse_data=minidom.parseString(data)
        #ght=parse_data.getElementsByTagName("ds")
        #print(ght.length)
        sentences=parse_data.getElementsByTagName("sentences")[0].getElementsByTagName("sentence")
        #ght=Element()
        #ght.TEXT_NODE
        #print(sentences[0].TEXT_NODE)
        i=0
        for sentence in sentences:
            nsentence={"text":"","aspects":[]}
            text=sentence.getElementsByTagName("text")[0]
            nsentence["text"]=self.replace_all_shorcuts(text.firstChild.nodeValue.lower())
            if with_punc==True:
                nsentence["text"]=self.replace_punctuaion(nsentence["text"],punctuation)
                nsentence["text"]=nsentence["text"].replace("'"," ")
                nsentence["text"]=" ".join(nsentence["text"].split())
            try:
                
                #now we find the aspecterms terms
                aspect_terms=sentence.getElementsByTagName("aspectTerms")
                for apt in aspect_terms:
                    aspects=apt.getElementsByTagName("aspectTerm")
                    for aspect in aspects:
                        n_aspect={"polarity":"neutral"}
                        n_aspect["term"]=self.replace_all_shorcuts(aspect.attributes["term"].value.lower())
                        n_aspect["term"]=n_aspect["term"].replace("'"," ")
                        n_aspect["term"]=" ".join(n_aspect["term"].split())
                        n_aspect["from"]=aspect.attributes["from"].value
                        n_aspect["to"]=aspect.attributes["to"].value
                        if(test==False):
                            n_aspect["polarity"]=aspect.attributes["polarity"].value.lower()
                        nsentence["aspects"].append(n_aspect)
                if(test==True):
                    self.test_sentences.append(nsentence)
                else:
                    self.sentences.append(nsentence)
                self.all_sentences.append(nsentence["text"])
            except Exception as e:
                print(nsentence["text"])
                print(i)
                print(e)
                break
                
            i=i+1
    
    def save_data_to_file(self,path):
        """Save the data like a pickle object in a file"""
        
        dict_data={"tokenizer":self.tokenizer,"sentences":self.sentences,"data":self.data,"max_length":self.max_length,"vocab_size":self.vocab_size,"all_sentences":self.all_sentences,"test_sentences":self.test_sentences,"rever_tokenizer":self.rever_tokenizer}
        with open(path,"wb") as f:
            pickle.dump(dict_data,f)
    
    
    def save_tokenizer_information(self,path):
        """Sauvegarde les informations sur le tokenizer"""
        dict_data={"tokenizer":self.tokenizer,"vocab_size":self.vocab_size,"max_length":self.max_length,"rever_tokenizer":self.rever_tokenizer}
        with open(path,"wb") as f:
            pickle.dump(dict_data,f)
    
    def load_tokenizer(self,path):
        with open(path,"rb") as f:
            dict_data=pickle.loads(f.read())
            self.tokenizer=dict_data["tokenizer"]
            self.max_length=dict_data["max_length"]
            self.vocab_size=dict_data["vocab_size"]
            self.rever_tokenizer=dict_data.get("rever_tokenizer",{})
    
    
    def load_data_from_file(self,path):
        """Load picke object data from file"""
        with open(path,"rb") as f:
            dict_data=pickle.loads(f.read())
            self.tokenizer=dict_data["tokenizer"]
            self.max_length=dict_data["max_length"]
            self.vocab_size=dict_data["vocab_size"]
            self.data=dict_data["data"]
            self.sentences=dict_data["sentences"]
            self.test_sentences=dict_data.get("test_sentences",[])
            self.all_sentences=dict_data.get("all_sentences",[])
            self.rever_tokenizer=dict_data.get("rever_tokenizer",{})
    
    def replace_punctuaion(self,text,ponctuation):
        """cette fonction cree de l'espace autour des ponctuations""" 
        text=str(text)
        for elt in ponctuation:
            text=text.replace(elt," "+elt+" ")
        return text
    
    def remove_punctuation(self,text,ponctuation):
        """Cette fonction supprimme la ponctuation 
        passe en parametre dans le text"""
        text=str(text)
        for elt in ponctuation:
            text=text.replace(elt," ")
        return text
    def create_tokenizer(self,with_punc=False):
        """A partirs de la liste des phrases on va creer un vocabulaire
        Dans un objet tokenizer de keras
        @with_punc permet de savoir si la ponctuation sera ignorer ou"""
        if with_punc==True:
            filters="\t\n"
            tokenizer=Tokenizer(filters=filters)
        else:
            tokenizer=Tokenizer()
        tokenizer.fit_on_texts(self.all_sentences)
        self.vocab_size=len(tokenizer.word_index)+1
        self.tokenizer=tokenizer
        self.rever_tokenizer=dict(map(reversed, tokenizer.word_index.items()))
        return tokenizer
    
    def set_sentences_max_length(self):
        """
        return and et the lenght of longeur sentence per word
        """
        self.max_length=max([len(self.tokenizer.texts_to_sequences([sentence])[0]) for sentence in self.all_sentences])
        
        return self.max_length
    
    def encode_text_to_vect(self,texts):
        """Use tokernizer to create a vector of entry text
            param @texts is a list of sentence only text
            return @sequence by using @self.max_length
        """
        sequence=self.tokenizer.texts_to_sequences(texts)
        
        #print(sequence)
        sequence=pad_sequences(sequences=sequence,maxlen=self.max_length,padding="post", truncating="post")
        return sequence
    
    def get_aspect_position(self,text,aspect_term,from_, to_):
        """cette fonction prends en entrer une phrase et deux indices et 
           retourne les indices de cette deniere"""
        aspects=aspect_term.split(" ")
        #print(aspects)
        text_token=text.split()
        aspects_position=[]
        j=0 #contient le debut de chaque mot en tant que indice dans la phrase
        i=0 #compteur dans la boucle
        start_index=0
        exist=False
        #print(from_)
        for wor in text_token:
            #print(j)
            if(j==from_ or j==(from_-1)):
                start_index=i
                exist=True
                break
            else:
                j=j+len(wor)+1
            i=i+1
        if exist==True:
            for aspect in aspects:
                aspects_position.append(start_index)
                start_index=start_index+1
        return aspects_position
    
    def get_aspect_position_2(self,text,aspect_term,begin):
        i=0
        j=0
        start_index=0
        exist=False
        aspects_index=[]
        ponct2={car:True for car in punctuation}
        ponct2[" "]=True
        #print(ponct2)
        for car in text:
            #print(car)
            test_car=ponct2.get(car,None)
            #print(test_car)
            if test_car:
                i=i+1
            if(j==begin):
                #print(i)
                #print(j,begin)
                start_index=i
                exist=True
            j=j+1
        if exist==True:
            for aspect in aspect_term.split():
                aspects_index.append(start_index)
                start_index=start_index+1
        return aspects_index
    
    def replace_all_shorcuts(self,text):
        """
           cette fonction remplace les raccourcis d'usage utilise en anglais. Nous pensons que cela peut 
           compresser de l'information et faire apprendre au model plus que necessaire
        """
        shorcuts={"'s":" is","i've":"i have","'ll":" will","n't":" not","'re":" are","'d":" had","'m":" am","'ve":" have"}
        text=text.lower()
        for elt in shorcuts:
            text=text.replace(elt,shorcuts[elt])
        return text
    
    def get_aspect_position_3(self,text,aspect_term,begin):
        """Cette fonction retourne la position d'un aspect en tant que mot 
        dans une liste de mot"""
        
        text_tokens=text_to_word_sequence(text.lower(), filters=self.tokenizer.filters)
        aspect_tokens=text_to_word_sequence(aspect_term.lower(), filters=self.tokenizer.filters)
        new_text=" ".join(text_tokens)
        new_aspect=" ".join(aspect_tokens)
        new_aspect=new_aspect.replace("'","")
        new_text=new_text.replace("'","")
        
        #all_occure=re.finditer(new_aspect,new_text)
        list_index=find_list_in_list(aspect_tokens,text_tokens)
        try:
            index_0=list_index[0]
        except Exception as e:
            try:
                new_aspect=self.replace_punctuaion(aspect_term,string.punctuation)
                new_text=new_text.replace("'","")
                text_tokens=text_to_word_sequence(new_text,filters=self.tokenizer.filters)
                aspect_tokens=text_to_word_sequence(new_aspect, filters=self.tokenizer.filters)
                
                list_index=find_list_in_list(aspect_tokens,text_tokens)
                
                index_0=list_index[0]
            except Exception as f:
                new_aspect=self.replace_punctuaion(aspect_term,string.punctuation)
#                new_text=new_text.replace("'","")
                text_tokens=text_to_word_sequence(new_text,filters=self.tokenizer.filters)
                aspect_tokens=text_to_word_sequence(new_aspect, filters=self.tokenizer.filters)
                print(" ".join(text_tokens))
                print("\n",text,"\n")
                print("\n --- \n")
                print(" ".join(aspect_tokens))
                print("\n\n")
                sys.exit()
        #print(len(list_index))
        if len(list_index)>1:
            
            #on retrouve l'index le plus proche de begin
            begin_distances=[]
            for elt in list_index:
                length=len(" ".join(text_tokens[0:elt+1]))
                begin_dist=abs(begin-length)
                begin_distances.append(begin_dist)
            min_dist_index=begin_distances.index(min(begin_distances))
            index_0=list_index[min_dist_index]
        return list(range(index_0,index_0+len(aspect_tokens)))    
        
        
        
        
    def get_ibo_vector(self,sentence,with_punc=False):
        """take sentence in parameter which is dict struture like that
            {"text":text,"aspects":[{"term":term,"from":begin_index,"to":end_index},...]}
            and return the ibo vector of this sentence
        """
        ibo_vector=np.zeros(self.max_length, dtype="int")
        #print(sentence["text"])
        aspect_index=[]
        for aspect in sentence["aspects"]:
            #print(aspect["term"])
            aspect_index=self.get_aspect_position_3(sentence["text"],aspect["term"],int(aspect["from"]))
            i=0
            for index in aspect_index:
                if(i==0):
                    ibo_vector[index]=1.0
                else:
                    ibo_vector[index]=2.0
                i=i+1
        if with_punc==True:
            text_tokens=text_to_word_sequence(sentence["text"],filters=self.tokenizer.filters)
            j=0
            for elt in text_tokens:
                if elt in string.punctuation and elt!="-":
                    ibo_vector[j]=3.0
                j=j+1
        
        return ibo_vector
    
            
    def create_train_data(self,test=False,nb_classe=3,with_punc=False):
        """Cette fonction va mettre toute les phrases sous forme de sequence 
            et faire un one hot encoding des mots"""
        train_data_x=[]
        train_data_y=[]
        real_ibo=[]
        if(test==False):
            sentences=[sentence["text"] for sentence in self.sentences]
        else:
            sentences=[sentence["text"] for sentence in self.test_sentences]
        for sentence in self.sentences:
            ibo=self.get_ibo_vector(sentence,with_punc)
            real_ibo.append(ibo)
            train_data_y.append(to_categorical(ibo,nb_classe))
        #print(sentences)
        train_data_x=np.array(self.encode_text_to_vect(sentences))
        #train_x=[]
        #for x in train_data_x:
        #    train_x.append(to_categorical(x,self.vocab_size))
        #train_data_x=np.array(train_x)
        return {"train_x":train_data_x,"train_y":np.array(train_data_y),"real_ibo":real_ibo}
    def save_object(self,object_to_save,path):
        with open(path,"wb") as f:
            pickle.dump(object_to_save,f)
    
    def load_object(self,path):
        with open(path,"rb") as f:
            return pickle.loads(f.read())
    
    def create_test_data(self,with_punc=False):
        test_data_x=[]
        test_data_y=[]
        sentences=[sentence["text"] for sentence in self.test_sentences]
        i=0
        for sentence in self.test_sentences:
            try:
                test_data_y.append(self.get_ibo_vector(sentence,with_punc))
            except Exception as e:
                print(sentence)
                print(i)
            i=i+1
        test_data_x=np.array(self.encode_text_to_vect(sentences))
        return {"test_x":test_data_x,"test_y":np.array(test_data_y)}
     
    def save_test_data(self,data,path):
        with open(path,"wb") as f:
            pickle.dump(data,f)
    
    def load_test_data(self,path):
        with open(path,"rb") as f:
            data=pickle.loads(f.read())
        return data
    def get_aspects_term(self,sentence):
        pass
    
    def save_to_word2vecfortmat(self,dict_data,path,vect_dimension=300):
        """Sauvegarde le dictionnaire dans un format word2vec dans un fichier texte"""
        i=0
        list_embeddings=[]
        
        list_embeddings.insert(0,str(i)+" "+str(vect_dimension))
        final_write="\n".join(list_embeddings)
        with open(path,"w") as f:
            for w in dict_data:
                f.writelines(" ".join([w," ".join(dict_data[w].astype(str))]))

from google.colab import drive
drive.mount('/content/gdrive')

pre=Prepocessing()

class MyTranslator:
  translate_word={}
  pre=Prepocessing()
  def __init__(self):
    pass
  @classmethod
  def translate_sentence(cls,sentence:str):
    #on split la phrase en mot
    
    sentence=pre.clean_emoji(sentence)
    sentence=sentence.split()
    
    final_sentence=[]
    my_punct=string.punctuation+"»«'´’…‘”“,¨¦´ʼʽʾʿˀˁ͵ͺ΄ʻ·"
    for elt in sentence:
      if (elt=="ˁ"):
        final_sentence.append("ˀ")
      elif elt in my_punct:
        final_sentence.append(elt)
      else:
        elt_2=cls.translate_word.get(elt,None)
        if (elt_2==None):
          final_sentence.append(elt)
        else:
          final_sentence.append(elt_2)
    
    return " ".join(final_sentence)

class MyTranslatorModel:
  def __init__(self,model_ibo,translator):
    self.model=model_ibo
    self.translator=translator
  
  def translate(self):
    pass

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
pre=Prepocessing()

french_sentences=pre.load_txt_sentences("/content/gdrive/MyDrive/absa_carmel/data/french_data/tripadvisor_commentt_fr.txt")
all_fr_words={}
my_punct=string.punctuation+"»«'´’…‘”“,¨¦´ʼʽʾʿˀˁ͵ͺ΄ʻ·"
sentences_to_form_word_to_vect=[]
i=0
for elt in french_sentences:
  elt=elt.lower()
  elt=pre.replace_punctuaion(elt,my_punct)
  elt_2=pre.clean_emoji(elt)
  elt_s=elt_2.split()
  sentences_to_form_word_to_vect.append(elt_s)
  for elt2 in elt_s:
    if (elt2.strip()!=""):
      all_fr_words[elt2]=True
print(len(all_fr_words))
word_list=list(all_fr_words)
word_to_translate=word_list
  

In [ ]:
from gensim.models import Word2Vec

word_2vec_fr=Word2Vec(sentences_to_form_word_to_vect,min_count=1,iter=100,size=50)

In [ ]:
word_2vec_fr.wv.save_word2vec_format("/content/gdrive/MyDrive/absa_carmel/data/word_representation/perso_embedding_fr_3.w2v")

In [ ]:
new_fr_model={}
length_fr=0
for elt in model_fr.wv.vocab:
  #print(elt)
  new_fr_model[elt]=elt+" "+(" ".join(model_fr[elt].astype(str)))
  length_fr+=1

for elt in word_2vec_fr.wv.vocab:
  exist_gt=new_fr_model.get(elt,None)
  if (exist_gt==None):
    new_fr_model[elt]=elt+" "+(" ".join(word_2vec_fr[elt].astype(str)))
    length_fr+=1
    
with open("/content/gdrive/MyDrive/absa_carmel/data/word_representation/cc.fr.50_2.vec","w") as f:
  first_line=str(length_fr)+" 50\n"
  f.write(first_line)
  for elt in new_fr_model:
    f.write(new_fr_model[elt]+"\n")


print("terminé....")


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


terminé....


In [ ]:

  break

ma


In [ ]:
model_en=FastText.load_fasttext_format("/content/gdrive/MyDrive/absa_carmel/data/word_representation/cc.en.50.bin")
model_fr=FastText.load_fasttext_format("/content/gdrive/MyDrive/absa_carmel/data/word_representation/cc.fr.50.bin")

In [ ]:
word_pairs=pre.load_object("/content/gdrive/MyDrive/absa_carmel/data/word_representation/train_words_pairs.bin")
j=0
new_word_pairs=[]
for elt in word_pairs:
  if (new_fr_model.get(elt[0],None)!=None):
    new_word_pairs.append(elt)
new_word_pairs.append(("dolé","food"))
new_word_pairs.append(("doumba","food"))
new_word_pairs.append(("ndolet","food"))

pre.save_object(new_word_pairs,"/content/gdrive/MyDrive/absa_carmel/data/word_representation/train_words_pairs.bin")

In [ ]:
print(len(word_pairs))
print(len(new_word_pairs))
print(len(all_fr_words))

8204
8204
9528


In [ ]:

#print(model_fr["doumba"])

In [ ]:
#creattion de la matrice de traduction
#model_en=KeyedVectors.load_word2vec_format("/content/gdrive/MyDrive/absa_carmel/data/word_representation/cc.en.50_2.vec")
model_fr_n=KeyedVectors.load_word2vec_format("/content/gdrive/MyDrive/absa_carmel/data/word_representation/cc.fr.50_2.vec")
#word_pairs=pre.load_object("/content/gdrive/MyDrive/absa_carmel/data/word_representation/train_words_pairs.bin")
#word_pairs.append(("dolé","food"))
#word_pairs.append(("doumba","food"))
#word_pairs.append(("ndolet","food"))
#translation_model=TranslationMatrix(model_fr,model_en,word_pairs=word_pairs)
#translation_model.train(word_pairs)
#translation_model.save("/content/gdrive/MyDrive/absa_carmel/model/translation_matrix/translation_matrix3.bin")
#translation_model=TranslationMatrix.load("/content/gdrive/MyDrive/absa_carmel/model/translation_matrix/translation_matrix3.bin")
test_punct="»«'´’…‘”“,¨¦´ʼʽʾʿˀˁ͵ͺ΄ʻ·"

In [ ]:
for pu in string.punctuation+test_punct:
  print(pu)
  try:
    model_en[pu]
  except Exception as e:
    print("u")
    try:
      model_fr_n[pu]
      print("***",pu," **** in fr")
    except Exception as e:
      print("*** ",pu," *** not in fr and en")


In [ ]:
#sentence="bonjour la famille je fais des expériences"
#rans=MyTranslator()
#MyTranslator.translator=translation_model
#result=MyTranslator.translate_sentence(sentence)
#result=[]
#MyTranslator.translator=[]
#print(result)


In [ ]:
#maintenant nous allons traduire toutes les phrases francaises en anglais
french_sentences=pre.load_object("/content/gdrive/MyDrive/absa_carmel/data/french_data/all_sentences.bin")
with open("/content/gdrive/MyDrive/absa_carmel/data/french_data/tripadvisor_commentt_fr.txt","r") as f:
  data=f.read()
  french_sentences=data.split()

traduction_en=[]
all_fr_words={}
my_punct=string.punctuation+"»«'´’…‘”“,¨¦´ʼʽʾʿˀˁ͵ͺ΄ʻ·"
sentences_to_form_word_to_vect=[]
i=0
for elt in french_sentences:
  elt=elt.lower()
  elt=pre.replace_punctuaion(elt,my_punct)
  elt_2=pre.clean_emoji(elt)
  elt_s=elt_2.split()
  sentences_to_form_word_to_vect.append(elt_s)
  for elt2 in elt_s:
    if (elt2.strip()!=""):
      all_fr_words[elt2]=True
print(len(all_fr_words))
word_list=list(all_fr_words)
word_to_translate=word_list
#word_to_translate.remove("doumba")
#word_to_translate.remove("dolé")


9528


In [ ]:
for w in word_list:
  if "‘" in w:
    print(w)
#pre.save_object(word_list,"/content/gdrive/MyDrive/absa_carmel/data/french_data/all_word_list_fr.bin")

‘


In [ ]:
word_to_translate=pre.load_object("/content/gdrive/MyDrive/absa_carmel/data/french_data/all_word_list_fr.bin")
length=len(word_to_translate)
#word_traduction_fr_to_en=pre.load_object("/content/gdrive/MyDrive/absa_carmel/data/french_data/words_fr_to_en_2.bin")
#print(len(word_traduction_fr_to_en))

print(length)
print(word_to_translate[0])

9528
ma


In [ ]:
print(length)

10362


In [ ]:
#print(gthyu)
#word_traduction_fr_to_en=pre.load_object("/content/gdrive/MyDrive/absa_carmel/data/cross_lingual/words_fr_to_en_2.bin")
word_traduction_fr_to_en={}
for i in range(0,length,300):
  traductions=translation_model.translate(word_to_translate[i:i+300],topn=2)
  for elt in traductions:
    word_traduction_fr_to_en[elt]=traductions[elt][0]
  print(i)
  pre.save_object(word_traduction_fr_to_en,"/content/gdrive/MyDrive/absa_carmel/data/cross_lingual/words_fr_to_en_3.bin")


In [ ]:
print(len(word_traduction_fr_to_en))

10362


In [ ]:
#print(i)
#word_list=list(word_traduction_fr_to_en)
#print(word_list[i])
#print(len(word_to_translate))
#print(len(word_traduction_fr_to_en))

In [ ]:
#pre.save_object(word_traduction_fr_to_en,"/content/gdrive/MyDrive/absa_carmel/data/french_data/words_fr_to_en.bin")
#print(i)
#print(word_to_translate[2399])
#print(len(word_traduction_fr_to_en))
#list_w=list(word_traduction_fr_to_en)
#print(list_w[3872])

In [ ]:
print(list(range(2)))

[0, 1]


In [ ]:
#nous allons traduit tous les mots  francais en anglais
#word_traduction_fr_to_en=pre.load_object("/content/gdrive/MyDrive/absa_carmel/data/french_data/words_fr_to_en.bin")
#i=383
#for j in range(i,10364):
#  elt=gthyu[j]
#  if (elt.strip()!=""):
#    trad=result=MyTranslator.translate_sentence(elt)
#    word_traduction_fr_to_en[elt]=trad
#pre.save_object(word_traduction_fr_to_en,"/content/gdrive/MyDrive/absa_carmel/data/french_data/words_fr_to_en.bin")

In [ ]:
#pre.save_object(word_traduction_fr_to_en,"/content/gdrive/MyDrive/absa_carmel/data/french_data/words_fr_to_en.bin")
#print(len(word_traduction_fr_to_en))
#gthy=list(all_fr_words)


In [ ]:
#print(word_to_translate)

In [ ]:
french_sentences=pre.load_txt_sentences("/content/gdrive/MyDrive/absa_carmel/data/french_data/tripadvisor_commentt_fr.txt")
my_punct=string.punctuation+"»«'´’…‘”“,¨¦´ʼʽʾʿˀˁ͵ͺ΄ʻ·"

i=0
for elt in french_sentences:
  elt=elt.lower()
  elt=pre.replace_punctuaion(elt,my_punct)
  elt=pre.clean_emoji(elt)
  french_sentences[i]=" ".join(elt.split())
  i=i+1

In [ ]:
pre.save_object(french_sentences,"/content/gdrive/MyDrive/absa_carmel/data/french_data/all_french_sentences.bin")
french_sentences[0]
word_traduction_fr_to_en

In [ ]:
MyTranslator.translate_word=word_traduction_fr_to_en
nsentences=[]
old_sentence=[]
sentence_trad=[]
for sentence in french_sentences:
  #sentence=sentence.lower()
  #no_emoji=pre.clean_emoji(sentence)
  new_sentence=MyTranslator.translate_sentence(sentence)
  nsentences.append(new_sentence)
  old_sentence.append(sentence)

  sentence_trad.append(sentence+"\n"+new_sentence)

import json

In [ ]:
i=0
json_trad=[]
for elt in old_sentence:
  json_trad.append({"original":elt,"traduction":nsentences[i]})
  i=i+1
json_trad={"sentences":json_trad}

with open("/content/gdrive/MyDrive/absa_carmel/data/traduction_en_fr_3.json", "w",encoding='utf-8') as f:
  json.dump(json_trad,f,ensure_ascii=False)

In [ ]:
#maintenant nous devons ecrit cela dans un fichier excel
#import pandas as pd

